<a href="https://colab.research.google.com/github/vedsub/ViT/blob/main/ViT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import random
import timeit
import math
import numpy
import numpy as np
from torch import optim
from torch import nn
!pip install torchinfo
from torchinfo import summary
import torch.nn.functional as F
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Dataset
from dataclasses import dataclass
from torch.optim.lr_scheduler import CosineAnnealingLR
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
@dataclass
class vit_config:
    num_channels: int = 3
    batch_size:int = 16
    image_size: int = 224
    patch_size: int = 16
    num_heads:int = 8
    dropout: float = 0.0
    layer_norm_eps: float = 1e-6
    num_encoder_layers: int = 12
    random_seed: int = 42
    epochs: int = 30
    num_classes: int = 10
    learning_rate: float = 1e-5
    adam_weight_decay: int = 0
    adam_betas: tuple = (0.9, 0.999)
    embd_dim: int = (patch_size ** 2) * num_channels           # 768
    num_patches: int = (image_size // patch_size) ** 2         # 196
    device: str = "cuda" if torch.cuda.is_available() else "cpu"
    print(device)

cpu


In [4]:
config = vit_config

random.seed(config.random_seed)
numpy.random.seed(config.random_seed)
torch.manual_seed(config.random_seed)
torch.cuda.manual_seed(config.random_seed)
torch.cuda.manual_seed_all(config.random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

In [5]:
#a custom embedding layer that concatenates and returns the patch embeddings along with position embeddings for each patch of the input image.

In [6]:
class VisionEmbedding(nn.Module):
    def __init__(self, config: vit_config):
        super().__init__()

        self.config  = config
        self.patch_embedding = nn.Sequential(
            nn.Conv2d(
                in_channels=config.num_channels,
                out_channels=config.embd_dim,
                kernel_size=config.patch_size,
                stride=config.patch_size,
                padding="valid"
            ),
            nn.Flatten(start_dim=2)
        )

        self.cls_token = nn.Parameter(torch.randn(size=(1, 1, config.embd_dim)), requires_grad=True)
        self.pos_embeddings = nn.Parameter(torch.randn(size=(1, config.num_patches + 1, config.embd_dim)), requires_grad=True)
        self.dropout = nn.Dropout(p=config.dropout)

    def forward(self, x : torch.Tensor) -> torch.Tensor:
        cls_token = self.cls_token.expand(x.shape[0], -1, -1)

        patch_embd = self.patch_embedding(x).transpose(2,1)
        patch_embd = torch.cat([cls_token, patch_embd], dim=1)
        embd = self.pos_embeddings + patch_embd
        embd = self.dropout(embd)
        return embd

In [7]:
model = VisionEmbedding(config)
dummy_input = torch.randn(1, config.num_channels, config.image_size, config.image_size).to(config.device)
output = model(dummy_input)
print(f"Output shape: {output.shape}")
print(f"Output (first 5 elements): {output[0, :5, :5]}")

Output shape: torch.Size([1, 197, 768])
Output (first 5 elements): tensor([[-0.4828, -2.9331, -1.0430,  0.5191,  1.1593],
        [-0.5916,  0.7174, -0.8135, -0.0548,  0.7490],
        [-0.5815,  0.2315, -0.9726, -1.6018,  0.6856],
        [-0.2447,  0.3341,  0.1944, -0.9270,  0.7581],
        [-0.9925,  0.5160,  0.9445,  0.3006,  0.7511]],
       grad_fn=<SliceBackward0>)


In [8]:
class VisionAttention(nn.Module):
    def __init__(self, config:vit_config):
        super().__init__()

        self.embd_dim = config.embd_dim
        self.num_heads = config.num_heads
        self.dropout = config.dropout

        self.q_proj = nn.Linear(self.embd_dim, self.embd_dim)
        self.k_proj = nn.Linear(self.embd_dim, self.embd_dim)
        self.v_proj = nn.Linear(self.embd_dim, self.embd_dim)
        self.out_proj = nn.Linear(self.embd_dim, self.embd_dim)

    def forward(self, x : torch.Tensor) -> torch.Tensor:
        B, T, C = x.shape

        query = self.q_proj(x)
        key = self.k_proj(x)
        value = self.v_proj(x)

        query = query.view(B, T, self.num_heads, C//self.num_heads).transpose(1,2)
        key = key.view(B, T, self.num_heads, C//self.num_heads).transpose(1,2)
        value = value.view(B, T, self.num_heads, C//self.num_heads).transpose(1,2)

        attn_score = (query @ key.transpose(-2, -1)) * (1.0 / math.sqrt(key.size(-1)))
        attn_score = F.softmax(attn_score, dim=-1).to(query.dtype)

        attn_out = (attn_score @ value).transpose(1,2)
        attn_out = attn_out.reshape(B, T, C).contiguous()
        attn_out = self.out_proj(attn_out)
        attn_out = F.dropout(attn_out, p=self.dropout, training=self.training)

        return attn_out

In [9]:
class VisionMLP(nn.Module):
    def __init__(self, config:vit_config):
        super().__init__()

        self.layer1 = nn.Linear(config.embd_dim, 3 * config.embd_dim)
        self.layer2 = nn.Linear(3 * config.embd_dim, config.embd_dim)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.layer1(x)
        x = nn.functional.gelu(x, approximate="tanh")
        x = self.layer2(x)
        return x

In [10]:
class VisionEncoderLayer(nn.Module):
    def __init__(self, config: vit_config):
        super().__init__()

        self.embd_dim = config.embd_dim
        self.attn = VisionAttention(config)
        self.layer_norm1 = nn.LayerNorm(self.embd_dim, eps=config.layer_norm_eps)
        self.mlp = VisionMLP(config)
        self.layer_norm2 = nn.LayerNorm(self.embd_dim, eps=config.layer_norm_eps)

    def forward(self, x:torch.Tensor) -> torch.Tensor:
        x = x + self.attn(self.layer_norm1(x))
        x = x + self.mlp(self.layer_norm2(x))
        return x

In [11]:
class VisionEncoder(nn.Module):
    def __init__(self, config: vit_config):
        super().__init__()
        self.layers = nn.ModuleList([VisionEncoderLayer(config) for _ in range(config.num_encoder_layers)])

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        for layer in self.layers:
            x = layer(x)
        return x

In [12]:
class VisionTransformer(nn.Module):
    def __init__(self, config= vit_config):
        super().__init__()

        self.embedding = VisionEmbedding(config)
        self.encoder = VisionEncoder(config)
        self.mlp_head = nn.Sequential(
            nn.LayerNorm(config.embd_dim, eps=config.layer_norm_eps),
            nn.Linear(config.embd_dim, config.num_classes)
        )

    def forward(self, x:torch.Tensor ) -> torch.Tensor:
        x = self.embedding(x)
        x = self.encoder(x)
        x = self.mlp_head(x[:, 0, :])
        return x

In [13]:
vit = VisionTransformer(vit_config)
summary(model=vit,
        input_size=(16, 3, 224, 224),
        col_names= ["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings= ["var_names"]
    )

Layer (type (var_name))                       Input Shape          Output Shape         Param #              Trainable
VisionTransformer (VisionTransformer)         [16, 3, 224, 224]    [16, 10]             --                   True
├─VisionEmbedding (embedding)                 [16, 3, 224, 224]    [16, 197, 768]       152,064              True
│    └─Sequential (patch_embedding)           [16, 3, 224, 224]    [16, 768, 196]       --                   True
│    │    └─Conv2d (0)                        [16, 3, 224, 224]    [16, 768, 14, 14]    590,592              True
│    │    └─Flatten (1)                       [16, 768, 14, 14]    [16, 768, 196]       --                   --
│    └─Dropout (dropout)                      [16, 197, 768]       [16, 197, 768]       --                   --
├─VisionEncoder (encoder)                     [16, 197, 768]       [16, 197, 768]       --                   True
│    └─ModuleList (layers)                    --                   --                  